In [11]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [12]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [13]:
def save_excel(df,col):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb.to_excel('AllProjectList.xlsx')
    return dfgb

In [14]:
subprocess.Popen('ProjectTime.xlsx', shell=True)

In [15]:
df = get_frame('ProjectTime.xlsx')

In [16]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = bd.get_quality(df['Duration'])
df = df.join(df_quality)

36 0.91 33


In [17]:
df.tail(10)

,Project,ProjeDate,Duration,Quality
39,KaggleLearn,2018-08-10,1287.0,0.299022
40,SpotifyPlayGen,2018-08-22,2178.0,0.516870
41,ZeroToDeepLearn,2018-08-26,2405.0,0.572372
42,FindYourJoy,2018-08-28,1635.0,0.384108
43,CUDAproject,2018-09-01,2250.0,0.534474
44,SpotifyPlayGen,2018-09-03,2894.0,0.691932
45,FlaskMegaTut,2018-09-08,4154.0,1.000000
46,FlaskMegaTut,2018-09-08,3908.0,0.939853
47,FlaskMegaTut,2018-09-11,1919.0,0.453545
48,FlaskMegaTut,2018-09-12,2193.0,0.520538


In [18]:
df.sort_values('Quality', ascending=False).head(10)

,Project,ProjeDate,Duration,Quality
45,FlaskMegaTut,2018-09-08,4154.0,1.000000
46,FlaskMegaTut,2018-09-08,3908.0,0.939853
36,SpotifyPlayGen,2018-08-04,3260.0,0.781418
44,SpotifyPlayGen,2018-09-03,2894.0,0.691932
33,GoogleCrashCourse,2018-07-29,2880.0,0.688509
30,ZeroToDeepLearn,2018-07-22,2748.0,0.656235
15,SpecCurator,2018-06-21,2719.0,0.649144
29,ZeroToDeepLearn,2018-07-20,2620.0,0.624939
35,SpotifyPlayGen,2018-08-02,2584.0,0.616137
31,GoogleCrashCourse,2018-07-24,2520.0,0.600489


In [19]:
dfgb = save_excel(df,'Project')

In [20]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1)
dfgb.sort_values('donext', ascending=False)

,Project,count,mean,var,total,donext
3,FlaskMegaTut,4,0.728484,0.079077,46,1.217657
11,VueClass,2,0.350733,0.000151,46,1.042528
1,DjangoClass,2,0.326895,0.003917,46,1.018690
10,TrollSniffer,3,0.444417,0.000468,46,1.009265
9,SpotifyPlayGen,6,0.595640,0.015383,46,0.995048
0,CUDAproject,4,0.497433,0.001883,46,0.986606
4,GoogleCrashCourse,4,0.483435,0.041059,46,0.972608
8,SpecCurator,4,0.473594,0.019456,46,0.962767
12,ZeroToDeepLearn,6,0.543562,0.013069,46,0.942970
5,KaggleLearn,3,0.376773,0.010887,46,0.941621
